SVG Path instructions: https://www.w3schools.com/graphics/svg_path.asp

In [1]:
import re
import pandas as pd

import numpy as np
import math
import serial
import pygame
import time
from pygame.locals import QUIT

import shapes

In [2]:
# to keep track of how many parameters an instruction needs 
num_param = {"m": 2, "l": 2, "h": 1, "v": 1, "z": 0,
            "c": 6, "s": 4, "q": 4, "t": 2, "a": 7}

In [3]:
# split svg path into its individual components
def split_path(path):
    ins = ""
    
    while len(path) > 0:
        #print path
        
        # if the first element is a letter, it is the instruction
        if path[0].isalpha():
            ins = path[0]
            path = path[1:]
            path = path.strip()
        
        # get number of necessary parameters
        n = str(num_param[ins.lower()])

        # split into part for this instruction and rest
        res = re.search(r"((?:[\-\d\.e\+]+[\s\,]*){"+n+"})(.*)", path)
        
        # collect parameters and the remaining path
        params = res.group(1).strip()
        path = res.group(2).strip()
        
        # split parameters into individual numbers
        params = re.split(r"[,\s]", params)
        params = [float(p) for p in params if p != ""]
        
        if ins == "m":
            ins = "M"
        
        
        # reorder the parameters
        yield [ins] + params[-2:] + params[:-2]

In [4]:
def clean_df(df):
    # flip y coordinates (we want (0,0) in the bottom left)
    df[["y", "C1y", "C2y"]] = -df[["y", "C1y", "C2y"]]

    # convert everything to absolute paths
    df["is_abs"] = df["ins"].apply(lambda x: x.isupper())
    df["group"] = df["is_abs"].cumsum()

    # add the relative paths to the absolute paths
    df["P2x"] = df.groupby("group").x.cumsum()
    df["P2y"] = df.groupby("group").y.cumsum()

    # shift y coordinate such that the origin is in (0,0)
    df.loc[df["is_abs"], "C1y"] -= df["P2y"].min()
    df.loc[df["is_abs"], "C2y"] -= df["P2y"].min()
    df["P2y"] -= df["P2y"].min()

    # add 'from' coordinates such that every row in the DataFrame is stand-alone
    df["P1x"] = df["P2x"].shift(1)
    df["P1y"] = df["P2y"].shift(1)
    df.loc[0, "P1x"] = 0
    df.loc[0, "P1y"] = 0

    # make these absolute as well
    df.loc[-df["is_abs"], "C1x"] += df["P1x"]
    df.loc[-df["is_abs"], "C1y"] += df["P1y"]
    df.loc[-df["is_abs"], "C2x"] += df["P1x"]
    df.loc[-df["is_abs"], "C2y"] += df["P1y"]

    # make sure that the letters are in uppercase
    df["ins"] = df["ins"].str.upper()

    df = df[["ins", "P1x", "P1y", "P2x", "P2y", "C1x", "C1y", "C2x", "C2y"]]
    
    return df

In [5]:
def normalize_df(df, x0, y0, xlim, ylim, margin):
    # make sure that x and y start at xmin, ymin,
    # width, height is xmax, ymax
    # with a margin
    # make sure everything is larger than 0
    xmin = df[["P1x", "P2x", "C1x", "C2x"]].min().min()
    df[["P1x", "P2x", "C1x", "C2x"]] -= xmin

    ymin = df[["P1y", "P2y", "C1y", "C2y"]].min().min()
    df[["P1y", "P2y", "C1y", "C2y"]] -= ymin

    # make sure everything is less than xlim - 2*margin
    xmax = df[["P1x", "P2x", "C1x", "C2x"]].max().max()
    xmul = (xlim-2*margin) / xmax

    ymax = df[["P1y", "P2y", "C1y", "C2y"]].max().max()
    ymul = (ylim-2*margin) / ymax

    df[["P1x", "P1y", "P2x", "P2y", "C1x", "C1y", "C2x", "C2y"]] *= min(xmul, ymul)

    # shift everything up by margin
    df[["P1x", "P1y", "P2x", "P2y", "C1x", "C1y", "C2x", "C2y"]] += margin
    
    
    df[["P1x", "P2x", "C1x", "C2x"]] += x0
    df[["P1y", "P2y", "C1y", "C2y"]] += y0

    return df

In [6]:
def preview_objs(objs):
    pygame.init()

    canvas=pygame.display.set_mode((240,170),0,32)

    white = (255, 255, 255)
    black = (0, 0, 0)

    canvas.fill(white)

    for obj in objs:
        obj.draw(canvas, black)

    while True:
        for event in pygame.event.get():
            if event.type==QUIT:
                pygame.quit()
                return
        pygame.display.update()

In [7]:
def paths_to_df(paths, xmin = 0, ymin = 0, xlim = 100., ylim = 100., margin = 5.):
    df = pd.DataFrame()
    # loop over paths in svg
    for path in paths:
        df = df.append(path_to_df(path))

    df = clean_df(df) # make all coordinates absolute
    df = normalize_df(df, xmin, ymin, xlim, ylim, margin) # make everything within bounds
    
    return df

In [8]:
def path_to_df(path):
    records = split_path(path)
    cols = ["ins", "x", "y", "C1x", "C1y", "C2x", "C2y"]
    df = pd.DataFrame.from_records(records, columns = cols)
        
    return df

In [9]:
def df_to_objs(df):
    objs = []

    for _, row in df.iterrows():
        if row["ins"] == "M": # move
            P2 = shapes.Point(*row[["P2x", "P2y"]])
            objs.append(P2)
            
        elif row["ins"] == "L": # line
            P1 = shapes.Point(*row[["P1x", "P1y"]])
            P2 = shapes.Point(*row[["P2x", "P2y"]])
            l = shapes.Line(P1, P2)
            objs.append(l)
            
        elif row["ins"] == "C": # curve
            P1 = shapes.Point(*row[["P1x", "P1y"]])
            P2 = shapes.Point(*row[["P2x", "P2y"]])
            C1 = shapes.Point(*row[["C1x", "C1y"]])
            C2 = shapes.Point(*row[["C2x", "C2y"]])
            bez = shapes.CubicBezier(P1, C1, C2, P2)

            # split curve if there is an inflection point
            ip1, ip2 = bez.inflection_points()

            if (ip1 == -1) and (ip2 != -1): # they are ordered
                bez1, bez2 = bez.split_at(ip2)
                objs.append(bez1)
                objs.append(bez2)
            elif (ip1 != -1) and (ip2 != -1): # split twice
                bez1, bez2 = bez.split_at(ip1)
                ip2 = (1 - ip1) * ip2
                bez3, bez4 = bez2.split_at(ip2)
                objs.append(bez1)
                objs.append(bez3)
                objs.append(bez4)
            else:
                objs.append(bez)
            
    return objs

In [10]:
def objs_to_plotter(objs, s):
    # display objects and ask for confirmation
    preview_objs(objs)
    r = input("plot this? [y/N]")
    if r != "y":
        return
    
    # if response was 'y', plot it
    for obj in objs:
        for line in obj.plot_instructions():
            l = line + "\n"
            print('Sending: ' + l,)
            s.write(l.encode('utf-8')) # Send g-code block to grbl
            grbl_out = s.readline() # Wait for grbl response with carriage return
            print(grbl_out.strip())

In [11]:
def wake_up_serial(port, baud):
    # Open grbl serial port
    s = serial.Serial(port, baud)

    # Wake up grbl
    s.write(b"\r\n\r\n")
    time.sleep(3)   # Wait for grbl to initialize 
    s.flushInput()  # Flush startup text in serial input

    s.write(b"G21\n")   # units = mm
    print(s.readline().strip())
    s.write(b"F5000\n") # feed rate
    print(s.readline().strip())

    s.write(b"S0 M3\n") # pen slightly down
    print(s.readline().strip())

    s.write(b"S0 M5\n") # pen up
    print(s.readline().strip())
    
    return s

In [12]:
P = shapes.Point(10, 80)
S = shapes.SineWave(P = P, A = 10, n = 50)
bez = S.to_bezier3()

In [ ]:
# k_min = 0
# k2_min = 0
# mse_min = 1e10

# k_step = 1e-7
# k2_step = 1e-6
# mse_step = 1e-2

# P = shapes.Point(0, 0)

# for k in np.arange(0.205155, 0.20517, k_step):
#     for k2 in np.arange(1.33575, 1.3359, k2_step):
#         S = shapes.SineWave(P = P, A = 1, n = 1, k = k, k2 = k2)

#         bez = S.to_bezier2()

#         mse = 0
#         for t in np.arange(0, 1, mse_step):
#             P1 = bez.point_at(t)
#             P2 = S.point_at(P1.x)
#             mse += (P1.y - P2.y)**2

#         if mse < mse_min:
#             mse_min = mse
#             k_min = k
#             k2_min = k2

# print(k_min, k2_min, mse_min)

In [21]:
P = shapes.Point(10, 80)
S = shapes.SineWave(P = P, A = 10, n = 50)
P2 = shapes.Point(60, 80)
S2 = shapes.SineWave(P = P2, A = 20, n = 50)

In [22]:
s = wake_up_serial('/dev/tty.wchusbserial1420', 115200)

b'ok'
b'ok'
b'ok'
b'ok'


In [24]:
objs_to_plotter([P, S, S2], s)

plot this? [y/N]y
Sending: S0 M5

b'ok'
Sending: G01 X-10 Y80

b'ok'
Sending: S1000 M3

b'ok'
Sending: G03 X-14.28942 Y85.10755 I-103.12248 J-82.24936

b'ok'
Sending: G03 X-16.25 Y87.07107 I-17.10509 J-15.11891

b'ok'
Sending: G03 X-19.78329 Y89.395 I-9.80549 J-11.06021

b'ok'
Sending: G03 X-22.5 Y90 I-2.71671 J-5.79714

b'ok'
Sending: G03 X-25.21671 Y89.395 I-0.0 J-6.40214

b'ok'
Sending: G03 X-28.75 Y87.07107 I6.2722 J-13.38415

b'ok'
Sending: G03 X-30.71058 Y85.10755 I15.14451 J-17.08242

b'ok'
Sending: G03 X-35.0 Y80 I98.83306 J-87.35691

b'ok'
Sending: G02 X-39.28942 Y74.89245 I-103.12248 J82.24936

b'ok'
Sending: G02 X-41.25 Y72.92893 I-17.10509 J15.11891

b'ok'
Sending: G02 X-44.78329 Y70.605 I-9.80549 J11.06021

b'ok'
Sending: G02 X-47.5 Y70 I-2.71671 J5.79714

b'ok'
Sending: G02 X-50.21671 Y70.605 I-0.0 J6.40214

b'ok'
Sending: G02 X-53.75 Y72.92893 I6.2722 J13.38415

b'ok'
Sending: G02 X-55.71058 Y74.89245 I15.14451 J17.08242

b'ok'
Sending: G02 X-60.0 Y80 I98.83306 J87.35691

In [25]:
s.write(b"S0 M5\n") # pen up
print(s.readline())

s.write(b"G00 X0 Y0\n") # go home
print(s.readline())

time.sleep(3)   # wait for everything to finish 

# close serial port
s.close()

b'ok\r\n'
b'ok\r\n'


In [ ]:
# # read svg and find paths
# f = open("/Users/Bas/Desktop/db.svg", "r")
# svg = "".join(f.readlines())
# paths = re.findall(r"path\s*d=\"(.+?)\"", svg)

# df = paths_to_df(paths, xmin = 0, ymin = 0, xlim = 100., ylim = 100., margin = 5.)
# objs = df_to_objs(df)

# s = wake_up_serial('/dev/tty.wchusbserial1420', 115200)

# objs_to_plotter(objs, s)

# s.write(b"S0 M5\n") # pen up
# print(s.readline())

# s.write(b"G00 X0 Y0\n") # go home
# print(s.readline())

# time.sleep(3)   # wait for everything to finish 

# # close serial port
# s.close()